In [5]:
import os
import utils.CONSTANTS as CONST
from collections import defaultdict
from tabulate import tabulate, SEPARATING_LINE
import pandas as pd


class XYRun():

    def __init__(self, runs: pd.DataFrame) -> None:

        self.runs = self.scan_runs(runs)


    @staticmethod
    def scan_runs(runs: pd.DataFrame) -> dict:
        run_dict, postXY = {}, False
        for _id, info in runs.iterrows():

            if info['forDB']: 
                postXY = True
                key = "XY"
            else:
                try:
                    key = "post" if postXY else "pre"
                    if "open shutter" in info["comment"]:
                        key += "XY"
                    else: 
                        key += "DAQ"
                except TypeError: continue

            if run_dict.get(key, None) is None or not postXY:
                run_dict[key] = _id

        return run_dict
    

    def __repr__(self):
        return str(self.runs)

    
    def __getitem__(self, key):
        return self.runs.get(key, "")

class Campaign():
    
    def __init__(self, year: int, month: int) -> None:

        runlist = CONST.SCAN_PATH / f"config/calib_runlists/calib_runs_{year}-{month:02}.list"
        
        if not os.path.isfile(runlist):
            raise FileNotFoundError(f"{runlist} not found!\nAvailable campaigns are:\n" + 
                                    "\n".join(os.listdir(CONST.SCAN_PATH / "config/calib_runlists")))

        self.data = pd.read_csv(runlist,
                                names=["id","tel","step","date","source","mA","forDB","jobfile","comment"],
                                dtype=defaultdict(lambda: "str", step="int", mA="float", forDB="bool"),
                                skipinitialspace=True,
                                parse_dates=["date"],
                                index_col=0,
                                comment="#",
                                sep=";")
        
        self.runs = self.split_runlist(self.data)


    def __getitem__(self, tel) -> XYRun:
        return self.runs[tel.upper()]


    @staticmethod
    def split_runlist(runlist: pd.DataFrame) -> dict:

        run_dict = {}
        for _, info in runlist[runlist['forDB']].iterrows():
            
            run_dict[info['tel'].upper()] = XYRun(runlist[
                (runlist['date'] == info['date'])
                & (runlist['tel'] == info['tel'])
                ])
        
        return run_dict


    def __repr__(self) -> True:
        
        data = dict(sorted(self.runs.items()))
        table_data = []

        last = "LLLMLACOHE"
        for tel, runs in data.items():
            if tel[:-1] not in last:
                table_data.append(SEPARATING_LINE)
            
            table_data.append(
                [tel,
                 runs['preDAQ'],
                 runs['preXY'],
                 runs['XY'],
                 runs['postXY'],
                 runs['postDAQ']
                 ]
            )
            
            last = tel[:-1]

        return tabulate(table_data, disable_numparse=True,
                        headers=['tel', 'preDAQ', 'preXY', 'XY', 'postXY', 'postDAQ'])

    

In [6]:
test = Campaign(2024,11)
print(test)

tel    preDAQ    preXY    XY      postXY    postDAQ
-----  --------  -------  ------  --------  ---------
CO1    16426m19  16430    16431   16432     16433m19
CO2    16426m20  16427    16428   16429     16433m20
CO3    16418m21  16419    16420   16421     16425m21
CO4    16405m22  16414    16415   16416     16417m22
CO5    16405m23  16406    16407   16408     16417m23
CO6    16405m24  16411    16412   16413     16417m24
-----  --------  -------  ------  --------  ---------
HE1    6511m25   6521     6522    6523      6524m25
HE2    6511m26   6515     6516    6517
HE3    6511m27   6518     6519    6520      6524m27
-----  --------  -------  ------  --------  ---------
LA1    12783m13  12790    12791   12792     12793m13
LA2    12783m14  12787    12788   12789     12793m14
LA3    12783m15  12784    12785   12786     12793m15
LA4    12794m16  12795    12796   12797     12804m16
LA5    12794m17  12798    12799   12800     12804m17
LA6    12794m18  12801    12802   12803     12804m18
-----  

In [ ]:
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt

fIn = "/cr/data01/filip/xy-calibration/campaign_summary/calib_runs_2024-11_OLO_results.csv"

df = pd.read_csv(fIn, comment='#', index_col=0,
                     dtype=defaultdict(lambda: str, global_pixel=int, mirror=int, 
                                       local_pixel=int, col=int, row=int))
df.columns = df.columns.str.strip()  # remove space in keys
label = fIn
label = label.replace("calib_runs_", "").replace("_results", "")

# because there may be nans in the cal_xy col
# pandas parses it as a default string, need to convert
df["cal_xy"] = df['cal_xy'].str.strip()
df["cal_xy"] = df["cal_xy"].astype(float)
df["mirror"] = df["mirror"].astype(float)

dfTelMean = df.groupby(["telescope"]).mean(numeric_only=True).reset_index()
plt.plot(dfTelMean["mirror"], dfTelMean["cal_xy"], '.', label=label)

In [ ]:
df.dtypes

In [ ]:
dfTelMean

In [ ]:
import pandas as pd
from collections import defaultdict

def load_runlist(year_month: str) -> pd.DataFrame:
    data = pd.read_csv(
        f"/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_{year_month}.list",
        names=["id","tel","step","date","source","mA","forDB","jobfile","comment"],
        dtype=defaultdict(lambda: str, step="int", mA="float", forDB="bool"),
        index_col=0,
        comment="#",
        sep=";",
    )

    for col in ["source", "jobfile", "comment"]:
        data[col] = data[col].map(lambda x: x.strip().replace('"',''))

    return data

def get_xy_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        runlist['forDB']
        ]

def get_good_xy_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        runlist['forDB']
        & (runlist['step'] == 6)
        & (runlist['source'] == "OLO")
        & (runlist['mA'] == 15.9)
        & (runlist['comment'] == "")
        ]

def get_cal_a_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        (runlist['step'] == 0)
        & (runlist['source'] == "")
        & (runlist['mA'] == 0)
        ]

def get_cal_a_open_shutter_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        (runlist['step'] == 0)
        & (runlist['source'] == "")
        & (runlist['mA'] == 0)
        & (runlist['comment'].str.contains("open shutter"))
        ]

runlist=load_runlist('2024-11')

get_xy_runs(runlist)

In [ ]:
print(runlist['source'] == "OLO")

In [ ]:
runlist.iloc[0]['source']

In [ ]:
from utils.Auger.SD.Simulation import Simulation

In [ ]:
sim = Simulation(name='test_sim', offline='ds_forced_station_triggers', 
                 src='/cr/users/filip/Projects/SSDTriggerEfficiency/SdSimulationReconstructionUpgrade/',
                 PRIMARY='photon')

In [ ]:
# sim.run(191)

In [ ]:
sim.status()

In [ ]:
sim.process()

In [ ]:
sim.status()